# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


In [37]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings.utils import rgb_to_int
from xlwings.constants import AutoFillType
import numpy as np

In [38]:
xw.__version__

'0.24.9'

In [39]:
#!pip install xlwings==0.24.3

## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [40]:
wb = xw.Book('себестоимостьА_в1.xlsx')
sheet = wb.sheets['Рецептура']
sheet.range('G7:O10').value

[[0.2, 0.4, 0.02, 0.02, 0.01, 0.005, 0.1, None, None],
 [0.07, 0.25, 0.01, 0.06, 0.015, None, 0.06, 0.05, None],
 [0.12, 0.22, 0.005, None, 0.009, 0.005, None, 0.05, 0.03],
 [0.12, 0.23, 0.01, None, 0.015, None, None, None, 0.05]]

In [41]:
consumption = sheet.range('G7:O10').options(np.array).value
consumption

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [42]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [43]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price 

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [44]:
cost = (np.nan_to_num(consumption) * unit_price).sum(axis=1)
cost

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [45]:
sheet.range('T7:T10').options(transpose=True).value = cost
sheet.range('T6').value = 'Себестоимость'
wb.save()

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [46]:
sht = wb.sheets.active

In [47]:
xw.Range('T4:T6').color = (255, 192, 0)  #изменение цвета ячеек
xw.Range('T3').color = (255, 255, 0)
xw.Range('T7:T13').color = (255, 255, 167)

In [48]:
rng_val1 = xw.Range((3,20), (16,20))
rng_val1.columns[0].autofit()  #изменение ширины столбца

In [49]:
wb.sheets[0].range('T6').font.bold = True  #полужирный шрифт

In [50]:
xw.Range('T6').api.Font.Color = rgb_to_int((255, 255, 255))  #окрасить шрифт в ячейке в белый

In [51]:
for i in range(8,11):   #установка границ в новом столбце
    for j in range(3, 14):
        a = sht.range('T'+str(j))
        a.api.Borders(i).LineStyle = 1
        a.api.Borders(i).Weight = 2

In [52]:
sht.range('T4:T6').merge()  #объединение ячеек

In [53]:
sht.range('T6').api.HorizontalAlignment = -4108

4. Выполнить 3 с помощью "протягиваемых" формул.

In [54]:
from xlwings.constants import AutoFillType

In [55]:
sheet.range('V7:V10').formula = '=SUMPRODUCT(G7:O7, G14:O14)'

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [56]:
import pandas as pd

In [57]:
recipes = pd.read_csv('recipes_sample.csv', sep = ',', usecols = ['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])
reviews = pd.read_csv('reviews_sample.csv', sep = ',', index_col =  0)

In [58]:
reviews

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [59]:
recipes

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [64]:
import xlwings as xw
from xlwings.constants import AutoFillType
from xlwings import constants 
import numpy as np

In [ ]:
recipes_sample = recipes.sample(frac=0.05, random_state=1)
reviews_sample = reviews.sample(frac=0.05, random_state=1)

wb = xw.Book('recipes.xlsx')

wb.sheets['Sheet1'].name ='Отзывы'
wb.sheets.add('Рецепты')

recipes_sheet = wb.sheets['Рецепты']
reviews_sheet = wb.sheets['Отзывы']

recipes_sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value = recipes_sample
reviews_sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value = reviews_sample

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
arr = np.array(recipes_sample.minutes * 60)

recipes_sheet.range('G1').value = 'seconds_assign'
recipes_sheet.range('G2:G1501').options(transpose=True).value = arr

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
recipes_sheet.range('H1').value = 'seconds_formula'
recipes_sheet.range('H2:H1501').formula = '=C2:C1501 * 60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
sheet = wb.sheets['Рецепты']
sht = wb.sheets.active
wb.sheets[0].range('A1:H1').font.bold = True  #полужирный шрифт
sht.range('A1:H1').api.HorizontalAlignment = -4108  #выравнивание по центру
wb.save()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for i in range(2, 1502):
    if sheet.range('C'+str(i)).value < 5:
        xw.Range('C'+str(i)).color = (0, 176, 80)
    elif 5 <= sheet.range('C'+str(i)).value <= 10:
        xw.Range('C'+str(i)).color = (255, 255, 0)
    else:
        xw.Range('C'+str(i)).color = (255, 0, 0)
wb.save()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
recipes_sheet.range('I1').value = 'n_reviews'
recipes_sheet.range('I2:I1501').formula = '= COUNTIF(Отзывы!B:B;Рецепты!B2)'

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate(arr1, arr2):
    lst_res = []
    rows = reviews_sheet.range('A2:E6336').current_region.last_cell.row
    for row in range(rows-1):
        if (not arr1[row,3].value in range(0,5)):
            arr1[row,:].color = (255, 0, 0)
        elif not arr1[row,1].value in arr2.value:
            lst_res.append(arr1[row,1].value)
            arr1[row,:].color = (255, 0, 0)
    return lst_res

In [ ]:
arr8 = reviews_sheet.range('A2:E6336').options(np.array)
arr8_1 = recipes_sheet.range('B2:B1501').options(np.array)
len(validate(arr8, arr8_1)) #кол-во соответвующих строк

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
recipes_model = pd.read_csv('recipes_model.csv', sep = '\t')
recipes_model

In [ ]:
wb = xw.Book('recipes_model.xlsx')
wb.sheets['Sheet1'].name = 'Модель'
rec_model_sheet = wb.sheets['Модель']
rec_model_sheet.range('A2').options(pd.DataFrame, header=1, index=False, expand='table').value = recipes_model
wb.save()

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
wb = xw.Book('recipes_model.xlsx')
wb.sheets['Sheet1'].name = 'Модель'
rec_model_sheet = wb.sheets['Модель']
rec_model_sheet.range('A2').options(pd.DataFrame, header=1, index=False, expand='table').value = recipes_model
wb.save()

In [35]:
recipes_model = pd.read_csv('recipes_model.csv', sep = '\t')
recipes_model

,recipe,name,varchar(100),Y,Рецепт,Название рецепта,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,recipe,id,serial,N,Рецепт,ID рецепта,PK,NaN,NaN
1,recipe,minutes,int,Y,Рецепт,Длительность выполнения в минутах,NaN,NaN,NaN
2,recipe,contributor_id,int,Y,Рецепт,ID автора рецепта,FK,user,id
3,recipe,submitted,timestamp,Y,Рецепт,Дата добавления,NaN,NaN,NaN
4,recipe,n_steps,int,N,Рецепт,Кол-во шагов рецепта,NaN,NaN,NaN
5,recipe,description,text,N,Рецепт,Описание рецепта,NaN,NaN,NaN
6,recipe,n_ingredients,int,N,Рецепт,Кол-во ингредиентов в рецепта,NaN,NaN,NaN
7,review,id,serial,Y,Отзыв,ID отзыва,PK,NaN,NaN
8,review,user_id,int,Y,Отзыв,ID автора отзыва,FK,NaN,NaN
9,review,recipe_id,int,Y,Отзыв,ID рецепта,FK,recipe,id


In [ ]:
rec_model_sheet.range('J2:J18').formula = '=B2&" "&PROPER(C2)&" "&IF(G2="PK";"PRIMARY KEY";"")&" "&IF(G2="FK";"REFERENCES"&" "&H2&" "&I2;"")&" "&IF(AND(D2="Y"; OR(G2 ="PK"; G2 = "")); "NOT NULL";"")'

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [ ]:
sheet = wb.sheets['Модель']
sht = wb.sheets.active
wb.sheets[0].range('A1:J1').font.bold = True  #полужирный шрифт
xw.Range('A1:J1').color = (0, 204, 255)  #цвет
sheet.autofit(axis="columns")  #автоподбор ширины столбца
sheet.used_range.api.AutoFilter(Field:=1)  #автофильтр
wb.save()

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [39]:
df = pd.DataFrame()
df['Сущность'] = [i for i in range(1,17)]
df['Количество атрибутов'] = 9 - recipes_model.apply(lambda x: x.isnull().sum(), axis='columns')
df

,Сущность,Количество атрибутов
0,1,7
1,2,6
2,3,9
3,4,6
4,5,6
5,6,6
6,7,6
7,8,7
8,9,7
9,10,9


In [40]:
wb.sheets.add('Статистика')
statis = wb.sheets['Статистика']
statis.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value = df

In [41]:
sheet = wb.sheets.active
chart = sheet.charts.add()
chart.set_source_data(sheet.range('B1:B17'))
chart.api[1].Left = sheet.range('E2').left
chart.api[1].Top = sheet.range('E2').top
wb.save('recipes_model.xlsx')